In [9]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
def pads1(path_excel, sheet_name, skiprows, brand = False):
    """
    Lee una hoja específica de un archivo Excel y procesa los datos para análisis temporal de PADS
    Argumentos:
    path_excel: str, ruta al archivo Excel
    sheet_name: str, nombre de la hoja a leer
    skiprows: int, número de filas a omitir al leer el archivo
    brand: bool, si es True, considera que existen datos de Brand ADS en la hoja que no deben ser analizados
    
    Retorna:
    DataFrame procesado con las métricas y valores de PADS
    """

    df = pd.read_excel(path_excel, sheet_name = sheet_name, skiprows = skiprows)
    df = df.dropna(how = 'all').dropna(how = 'all', axis = 1).reset_index(drop = True)
    df = df.drop(columns = ['TOTALES'])

    if brand:
        df = df.iloc[0:14, :]

    metricas = df.iloc[::2].reset_index(drop = True)
    valores = df.iloc[1::2].reset_index(drop = True)

    valores['MÉTRICA'] = metricas['RDS1']
    valores.rename(columns = {'TOTALES': 'CONSOLIDADO'}, inplace = True)

    valores_long = valores.melt(
        id_vars = 'MÉTRICA',
        var_name = 'Cuenta Meli',
        value_name = 'Valor'
    )

    valores_final = valores_long.pivot(
        index = 'Cuenta Meli',
        columns = 'MÉTRICA',
        values = 'Valor'
    ).reset_index()
    valores_final.columns.name = None

    try:
        fecha = pd.to_datetime(sheet_name, format = '%B %y', errors = 'coerce', dayfirst = True)

        if pd.isna(fecha):
            meses = {
                "ENERO": "January", "FEBRERO": "February", "MARZO": "March",
                "ABRIL": "April", "MAYO": "May", "JUNIO": "June",
                "JULIO": "July", "AGOSTO": "August", "SEPTIEMBRE": "September",
                "OCTUBRE": "October", "NOVIEMBRE": "November", "DICIEMBRE": "December"
            }
            mes, año = sheet_name.split()
            mes_en = meses.get(mes.upper())
            fecha = pd.to_datetime(f"{mes_en} 20{año}", format="%B %Y")

    except:
        fecha = None

    valores_final['Fecha'] = fecha

    return valores_final

In [3]:
marzo23 = pads1(
    '/Users/martincarrasco/Desktop/Martín_Carrasco/Gestión PADS y Brand ADS/PADS MENSUALES/PADS MENSUALES 2025-07.xlsx',
    'MARZO 23',
    skiprows = 0
)

marzo23

,Cuenta Meli,ACOS,Clics,Impresiones,Ingresos,Inversión,Ventas por publicidad,Ventas sin publicidad,Fecha
0,BICISOL,0.0741,3023,766810,6377432,472530,128,94,2023-03-01
1,BLACKPARTS,0.0613,3458,2624886,18685124,1145593,129,96,2023-03-01
2,MERCADO REPUESTOS,0.0683,5772,2016203,11753632,803262,269,153,2023-03-01
3,RDS1,0.0659,8990,6783486,28507260,1879137,566,336,2023-03-01
4,RDS3,0.0685,2874,1885139,5923211,405704,120,228,2023-03-01
5,REICARS,0.0807,3022,2138525,6749266,544786,143,255,2023-03-01
6,TRIANA,0.067,5157,3906754,10121280,678125,252,201,2023-03-01


In [4]:
abril25 = pads1(
    '/Users/martincarrasco/Desktop/Martín_Carrasco/Gestión PADS y Brand ADS/PADS MENSUALES/PADS MENSUALES 2025-07.xlsx',
    'ABRIL 25',
    skiprows = 2,
    brand = True
)

abril25

,Cuenta Meli,ACOS,Clics,Impresiones,Ingresos,Inversión,Ventas por publicidad,Ventas sin publicidad,Fecha
0,BICISOL,0.0904,7543,3216237,11630553,1051827,302,147,2025-04-01
1,BLACKPARTS,0.1007,7807,3578842,21130344,2128061,218,73,2025-04-01
2,INDUSOL,0.1342,2146,1256582,2777499,372613,51,41,2025-04-01
3,MERCADO REPUESTOS,0.157,7643,4154914,8117631,1274620,282,111,2025-04-01
4,NEUMASOL,0.0863,1139,378873,4319393,372860,38,17,2025-04-01
5,RDS1,0.122,17063,8286561,39652215,4837938,999,529,2025-04-01
6,RDS3,0.1095,4313,2306607,7572794,829041,173,79,2025-04-01
7,REICARS,0.08,6272,2433230,13555702,1084927,330,142,2025-04-01
8,TRIANA,0.1197,3272,1448793,5307282,635046,197,136,2025-04-01


In [5]:
sheet_names1 = [
    'MARZO 23', 'ABRIL 23', 'MAYO 23', 'JUNIO 23', 'JULIO 23',
    'AGOSTO 23', 'SEPTIEMBRE 23', 'OCTUBRE 23', 'NOVIEMBRE 23', 'DICIEMBRE 23',
    'ENERO 24', 'FEBRERO 24', 'MARZO 24', 'ABRIL 24', 'MAYO 24',
    'JUNIO 24', 'JULIO 24', 'AGOSTO 24', 'SEPTIEMBRE 24', 'OCTUBRE 24',
    'NOVIEMBRE 24', 'DICIEMBRE 24', 'ENERO 25', 'FEBRERO 25', 'MARZO 25'
]

for sheet_name in sheet_names1:
    # Leer y procesar cada hoja de sheet_names1, luego concatenar todos los DataFrames procesados en un solo DataFrame, guardando el resultado en un csv
    df = pads1(
        '/Users/martincarrasco/Desktop/Martín_Carrasco/Gestión PADS y Brand ADS/PADS MENSUALES/PADS MENSUALES 2025-07.xlsx',
        sheet_name,
        skiprows = 0
    )
    df.to_csv(f'/Users/martincarrasco/Desktop/Martín_Carrasco/Gestión PADS y Brand ADS/PADS MENSUALES/AJUSTE PADS/PADS {sheet_name}.csv', index = False)
    print(f"{sheet_name} procesado y guardado en csv.")


MARZO 23 procesado y guardado en csv.
ABRIL 23 procesado y guardado en csv.
MAYO 23 procesado y guardado en csv.
JUNIO 23 procesado y guardado en csv.
JULIO 23 procesado y guardado en csv.
AGOSTO 23 procesado y guardado en csv.
SEPTIEMBRE 23 procesado y guardado en csv.
OCTUBRE 23 procesado y guardado en csv.
NOVIEMBRE 23 procesado y guardado en csv.
DICIEMBRE 23 procesado y guardado en csv.
ENERO 24 procesado y guardado en csv.
FEBRERO 24 procesado y guardado en csv.
MARZO 24 procesado y guardado en csv.
ABRIL 24 procesado y guardado en csv.
MAYO 24 procesado y guardado en csv.
JUNIO 24 procesado y guardado en csv.
JULIO 24 procesado y guardado en csv.
AGOSTO 24 procesado y guardado en csv.
SEPTIEMBRE 24 procesado y guardado en csv.
OCTUBRE 24 procesado y guardado en csv.
NOVIEMBRE 24 procesado y guardado en csv.
DICIEMBRE 24 procesado y guardado en csv.
ENERO 25 procesado y guardado en csv.
FEBRERO 25 procesado y guardado en csv.
MARZO 25 procesado y guardado en csv.


In [6]:
sheet_names2 = ['ABRIL 25', 'MAYO 25', 'JUNIO 25', 'JULIO 25']

for sheet_name in sheet_names2:

    df = pads1(
        '/Users/martincarrasco/Desktop/Martín_Carrasco/Gestión PADS y Brand ADS/PADS MENSUALES/PADS MENSUALES 2025-07.xlsx',
        sheet_name,
        skiprows = 2,
        brand = True
    )
    df.to_csv(f'/Users/martincarrasco/Desktop/Martín_Carrasco/Gestión PADS y Brand ADS/PADS MENSUALES/AJUSTE PADS/PADS {sheet_name}.csv', index = False)
    print(f"{sheet_name} procesado y guardado en csv.")

ABRIL 25 procesado y guardado en csv.
MAYO 25 procesado y guardado en csv.
JUNIO 25 procesado y guardado en csv.
JULIO 25 procesado y guardado en csv.


In [16]:
def concatenar_pads(carpeta):
    """
    Concatenar todos los archivos CSV de PADS en una carpeta específica en un solo DataFrame.
    
    Argumentos:
    path_carpeta: str, ruta a la carpeta que contiene los archivos CSV
    
    Retorna:
    DataFrame concatenado con todos los datos de PADS
    """

    archivos = glob.glob(os.path.join(carpeta, "PADS*.csv"))

    if not archivos:
        print("No se encontraron archivos CSV que comiencen con 'PADS'")
        return pd.DataFrame()

    dfs = []
    for archivo in archivos:
        df = pd.read_csv(archivo)
        dfs.append(df)

    # Concatenar todos
    df_final = pd.concat(dfs, ignore_index=True)
    df_final = df_final.sort_values(by = 'Fecha', ascending = True).reset_index(drop = True)
    return df_final

In [20]:
pads = concatenar_pads('/Users/martincarrasco/Desktop/Martín_Carrasco/Gestión PADS y Brand ADS/PADS MENSUALES/AJUSTE PADS')

pads.to_excel(
    '/Users/martincarrasco/Desktop/Martín_Carrasco/Gestión PADS y Brand ADS/PADS MENSUALES/AJUSTE PADS/DATA PADS MARZO 23 - JULIO 25.xlsx',
    index = False
)